In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/all-labelled/all_labelled.tsv


In [2]:
# Read in data
df=pd.read_csv("../input/all-labelled/all_labelled.tsv", delimiter="\t")

In [3]:
!pip install -q ktrain 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.20.3 which is incompatible.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.5.1 which is incompatible.
imbalanced-learn 0.9.0 requires scikit-learn>=1.0.1, but you have scikit-learn 0.24.2 which is incompatible.
hypertools 0.7.0 requires scikit-learn!=0.22,<0.24,>=0.19.1, but you have scikit-learn 0.24.2 which is incompatible.
featuretools 1.4.1 requires numpy>=1.21.0, but you have numpy 1.20.3 which is incompatible.


In [4]:
df['text']=df['abstract']
df['label']=df['class']
df.drop(['abstract', 'journal', 'title', 'uid', 'class'], axis=1, inplace=True)
df.dropna(inplace=True)
df=df.drop_duplicates()

In [5]:
import tensorflow as tf 
import ktrain 
from ktrain import text 
from sklearn.model_selection import train_test_split
# Enable AMP
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [6]:
df.dropna(inplace=True)
X = df['text'].tolist()
y = df['label'].tolist() 
pos = y.count(1)
neg=y.count(0)
total=len(y)
print(pos)
print(neg)
print(total)

988
7640
8628


In [7]:
t_mod = text.Transformer('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', maxlen=500, class_names = [0,1])

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import copy

In [9]:
#weight_for_0 = (1 / y.count(0)) * (len(y) / 2.0)
#weight_for_1 = (1 / y.count(1)) * (len(y) / 2.0)

#class_weight = {0: weight_for_0, 1: weight_for_1}
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
y=np.asarray(y)
X=np.asarray(X)

In [10]:
# Re-run cell if one of the classifiers predicts all 0s

# Initialize empty array for prediction probabilities/confidence
psx = np.zeros((len(y), 2))

# Fill in this array using k fold cross validation 
for k, (cv_train_idx, cv_holdout_idx) in enumerate(kf.split(X, y)):
    t_mod = text.Transformer('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', maxlen=500, class_names = [0,1])

    # Select the training and holdout cross-validated sets.
    X_train_cv, X_holdout_cv = X[cv_train_idx], X[cv_holdout_idx]
    y_train_cv, y_holdout_cv = y[cv_train_idx], y[cv_holdout_idx]
    # Preprocess train 
    data= t_mod.preprocess_train(X_train_cv,y_train_cv)
    
    # Get new classifier for each iteration
    model = t_mod.get_classifier()
    
    # Fit model
    learner = ktrain.get_learner(model, train_data=data, batch_size=16)
    learning_rate= 5e-5
    epochs=3
    learner.fit_onecycle(learning_rate, epochs)
    predictor=ktrain.get_predictor(learner.model, preproc=t_mod)
    
    
    # Check performance of model 
    predictions=predictor.predict((X_holdout_cv))
    print(accuracy_score((y_holdout_cv), predictions))
    print(classification_report((y_holdout_cv), predictions))

    
    # Get probabilities
    psx_cv = predictor.predict_proba(X_holdout_cv)  # P(s = k|x) # [:,1]
    psx[cv_holdout_idx] = psx_cv


preprocessing train...
language: en
train sequence lengths:
	mean : 170
	95percentile : 249
	99percentile : 271


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

404 Client Error: Not Found for url: https://huggingface.co/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext/resolve/main/tf_model.h5


Is Multi-Label? False


...



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/3
432/432 [==============================] - 450s 993ms/step - loss: 0.2406 - accuracy: 0.8977
Epoch 2/3
432/432 [==============================] - 431s 993ms/step - loss: 0.1827 - accuracy: 0.9179
Epoch 3/3
432/432 [==============================] - 430s 993ms/step - loss: 0.0961 - accuracy: 0.9613
0.9153623188405797
              precision    recall  f1-score   support

           0       0.96      0.95      0.95      1528
           1       0.62      0.66      0.64       197

    accuracy                           0.92      1725
   macro avg       0.79      0.80      0.80      1725
weighted avg       0.92      0.92      0.92      1725

preprocessing train...
language: en
train sequence lengths:
	mean : 171
	95percentile : 249
	99percentile : 266


404 Client Error: Not Found for url: https://huggingface.co/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext/resolve/main/tf_model.h5


Is Multi-Label? False


/opt/conda/lib/python3.7/site-packages/ktrain/text/preprocessor.py:1070: UserWarning: Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  warnings.warn('Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. ' +\




begin training using onecycle policy with max lr of 5e-05...
Epoch 1/3
432/432 [==============================] - 451s 992ms/step - loss: 0.2476 - accuracy: 0.8942
Epoch 2/3
432/432 [==============================] - 430s 992ms/step - loss: 0.1829 - accuracy: 0.9124
Epoch 3/3
432/432 [==============================] - 431s 993ms/step - loss: 0.1102 - accuracy: 0.9507
0.9147826086956522
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      1528
           1       0.61      0.72      0.66       197

    accuracy                           0.91      1725
   macro avg       0.78      0.83      0.81      1725
weighted avg       0.92      0.91      0.92      1725



In [11]:
!pip install -q cleanlab

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
from cleanlab.pruning import get_noise_indices

ordered_label_errors = get_noise_indices(
    s=y,
    psx=psx,
    sorted_index_method='normalized_margin', # Orders label errors
 )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
print(len(ordered_label_errors))
print(ordered_label_errors)

509
[7562 2752  165 4701 3843 1857 2800 1404 3650 4125 2499 5258 2751 1542
  492 7515 3799 2766 3662 4565 3375 2724 4136   57 2886 3818 1321 2097
 4713 1515 4690 7586 1821  557 8156 8305 2632 8342 7920 3991 1226 3773
 4124 4018 1041 1302 4193 3794 3699  976  205 3802 1735 3944 2662 1015
 2656 1629 1964 7923 1728  951 6426  987 5554 2120 1928 4564  325 5118
 3174 2787 6450 3803 5490 2697 3518 7078 1068 1002 5530 1307 4148 4471
 4536 1759 2767 2378 3865 1413 1552 3630  994 8386 7786 4418 4827 3187
 8146 1174 4681 4375  849 2599 1854  190 5138 2224 1230 4469 1613 3471
 1914  107  700 4722 3327 4260 2758 5322 2208 1416 4446  114 3258 2940
 2400 5585 3018 1282 1272 2056 5408 2199 1705 2107 2467 1711 4555  710
 2139 1284  397  250 1704 3738  152 1156  461 5946 5489 4918 7567   63
 5292 2567 3517 1968  401  835 2765 4752  241 1582 1421 3084 5360 4712
 2368 2187 3550  318  255 2277 1179  433 2449  436  352 3221 1676 4494
 4627  749  883 2821 2900 6456  599 6432 5392  274  664   77  296  782
  

In [14]:
list_abs=[]
for i in ordered_label_errors: 
    list_abs.append(X[i])
print(len(list_abs))

509


In [15]:
df_o=pd.read_csv("../input/all-labelled/all_labelled.tsv", delimiter="\t")
df_o.dropna(inplace=True)
df_o=df_o.drop_duplicates()
df_top_errors= df_o[df_o['abstract'].isin(list_abs)]
print(len(df_top_errors))

529


In [16]:
print(df_top_errors)


           uid                                           abstract  \
7     31969690  Neonatal mammalian heart maintains a transient...   
20    32127658  Autophagy is a cellular catabolic process that...   
29    32015503  Cells experiencing delays in mitotic progressi...   
50    32376875  We have previously reported that Monoglyceride...   
60    32332916  Although the roles of the Hippo pathway in org...   
...        ...                                                ...   
8442  33591274  Chromosome segregation during cell division re...   
8458  33587037  Piwi-interacting RNAs (piRNAs) play essential ...   
8491  33555257  The Hippo (Hpo) pathway regulates tissue growt...   
8613  33459596  Germline mutations in the Folliculin (FLCN) tu...   
8669  33416496  The oncoprotein transcription factor MYC is a ...   

                                                  title  \
7     Targeting LncDACH1 promotes cardiac repair and...   
20    Deleting key autophagy elongation proteins ind.

In [17]:
df_top_errors.to_csv('top_losses_cl.tsv', sep= "\t", index=False)